In [50]:
from nltk.stem import PorterStemmer
import numpy as np

In [51]:


class TextProcessor:
    def __init__(self):
        self.stemmer = PorterStemmer()
        self.stoplist_words = set()

    def load_stopwords(self, stoplist_file):
        with open(stoplist_file, 'r') as stoplist:
            for word in stoplist:
                self.stoplist_words.add(self.stemmer.stem(word.strip()))

    def process_documents(self, document_files):
        total_words = set()
        document_sets = []

        for document_file in document_files:
            temp = set()
            with open("docs/"+document_file, 'r') as file:
                for line in file:
                    words = line.split()
                    for word in words:
                        if word[-1] in [".", ",", ":", ";", "?", "!", "¡", "¿"]:
                            word = word[:-1]
                        word = self.stemmer.stem(word.lower())
                        if word.lower() not in self.stoplist_words:
                            total_words.add(word)
                            temp.add(word)
            document_sets.append(temp)

        return total_words, document_sets



In [52]:
import math
from collections import defaultdict

class TFIDFProcessor:
    def __init__(self):
        self.tf = defaultdict(lambda: defaultdict(int))
        self.idf = {}
        self.tfidf = defaultdict(lambda: defaultdict(float))
        self.total_documentos = 0

    def _calcular_tf(self, conjunto_documentos):
        for i, conjunto in enumerate(conjunto_documentos):
            total_palabras = len(conjunto)
            for palabra in conjunto:
                self.tf[palabra][i] = 1 / total_palabras

    def _calcular_idf(self, conjunto_documentos):
        conjunto_total = set()
        for conjunto in conjunto_documentos:
            conjunto_total.update(conjunto)
        self.total_documentos = len(conjunto_documentos)

        for palabra in conjunto_total:
            documentos_con_palabra = sum(1 for i in range(self.total_documentos) if self.tf[palabra][i] > 0)
            self.idf[palabra] = math.log(self.total_documentos / documentos_con_palabra)

    def calcular_tfidf(self, conjunto_documentos):
        self._calcular_tf(conjunto_documentos)
        self._calcular_idf(conjunto_documentos)

        for palabra in self.idf.keys():
            for i in range(self.total_documentos):
                self.tfidf[palabra][i] = self.tf[palabra][i] * self.idf[palabra]
        return self.tfidf

    def obtener_vectores_tfidf(self, conjunto_documentos):
        self.calcular_tfidf(conjunto_documentos)
        vectores_tfidf = []
        for i in range(self.total_documentos):
            vector = {palabra: self.tfidf[palabra][i] for palabra in self.tfidf}
            vectores_tfidf.append(vector)
        return vectores_tfidf




In [110]:
# Uso de la clase TextProcessor
text_processor = TextProcessor()

# Cargar stopwords desde el archivo 'stoplist.txt'
text_processor.load_stopwords('stoplist.txt')

# Leer y procesar los documentos
libros = ["libro1.txt", "libro2.txt", "libro3.txt", "libro4.txt", "libro5.txt", "libro6.txt","libro7.txt"]
# libros = ["test1.txt","test2.txt","test3.txt"]
query = ["query.txt"]
total_words, conjuntos_libros = text_processor.process_documents(libros)
query_words, conjunto_query = text_processor.process_documents(query)
# print("Stopwords:", text_processor.stoplist_words)
# print("Total de palabras:", total_words)
# print("Conjuntos de palabras por documento:", conjuntos_libros)

tfidf_processor = TFIDFProcessor()
tfidf_resultados = tfidf_processor.calcular_tfidf(conjuntos_libros)
query_resultados = tfidf_processor.calcular_tfidf(conjunto_query)
# # Impresión de los pesos TF-IDF
# for palabra, valores in tfidf_resultados.items():
#     print(f"Palabra: {palabra}")
#     for i, peso in valores.items():
#         print(f"  Documento {i+1}: TF-IDF = {peso:.3f}")

vectores_tfidf_textos = tfidf_processor.obtener_vectores_tfidf(conjuntos_libros)
vectores_tfidf_query = tfidf_processor.obtener_vectores_tfidf(conjunto_query)
# Impresión de los vectores TF-IDF
for i, vector in enumerate(vectores_tfidf_textos):
    print(f"tfidf_doc{i + 1} =", vector)

for i, vector in enumerate(vectores_tfidf_query):
    print(f"tfidf_query =", vector)

print(query_words)
print(conjunto_query)
print(query_resultados)

tfidf_doc1 = {'salir': 0.006808494393996565, 'oculto': 0.0, 'espesa': 0.0, 'maldición': 0.0, 'gandalf': 0.0008377754338437954, 'lengua': 0.0, 'caía': 0.0, 'reún': 0.0, 'representant': 0.0, 'ciento': 0.0, 'edora': 0.0, 'librars': 0.0, 'gigant': 0.0, 'ordena': 0.010575598636170181, 'aniquilado': 0.0, 'ponerlo': 0.0, 'rohan': 0.0, 'peripecia': 0.0, 'culmina': 0.0, 'derribada': 0.0, 'contó': 0.0, 'desconcierto': 0.0, 'descend': 0.0, 'princip': 0.010575598636170181, 'pretexto': 0.010575598636170181, 'derrotan': 0.0, 'escena': 0.0, 'descubren': 0.0, 'retractara': 0.0, 'capturado': 0.004604879676017411, 'perseguido': 0.010575598636170181, 'sabio': 0.010575598636170181, 'enorm': 0.0, 'pago': 0.0, 'cantar': 0.010575598636170181, 'inició': 0.0, 'rohirrim': 0.0, 'seguía': 0.0, 'dormían': 0.0, 'brujo': 0.0, 'ella-laraña': 0.0, 'custodiándola': 0.0, 'gamo': 0.010575598636170181, 'escapars': 0.0, 'oportuna': 0.0, 'faramir': 0.0, 'lomo': 0.0, 'encerró': 0.0, 'accidentalment': 0.010575598636170181, 'm

In [87]:
def cosine_similarity(vector1, vector2):
    # Convertir los diccionarios en listas ordenadas de valores (para garantizar el mismo orden)
    values1 = [vector1[word] for word in sorted(vector1)]
    values2 = [vector2[word] for word in sorted(vector2)]

    # Crear vectores NumPy a partir de las listas de valores
    vector1 = np.array(values1)
    vector2 = np.array(values2)

    # Calcular la similitud del coseno utilizando NumPy
    dot_product = np.dot(vector1, vector2)
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)

    if norm_vector1 == 0 or norm_vector2 == 0:
        return 0  # Evitar la división por cero
    else:
        return dot_product / (norm_vector1 * norm_vector2)


In [101]:

similitud = cosine_similarity(vectores_tfidf_textos[1], vectores_tfidf_textos[1])
num_documentos = len(vectores_tfidf_textos)

# Calcular la similitud del coseno para todos los pares de documentos y mostrar con tres decimales
for i in range(num_documentos):
    for j in range(num_documentos):
        similarity = cosine_similarity(vectores_tfidf_textos[i], vectores_tfidf_textos[j])
        print("{:.3f}".format(similarity), end=" ")  # Formatear y mostrar con tres decimales
    print()


1.000 0.033 0.017 0.025 0.026 0.038 0.038 
0.033 1.000 0.029 0.031 0.028 0.029 0.029 
0.017 0.029 1.000 0.028 0.039 0.020 0.020 
0.025 0.031 0.028 1.000 0.038 0.033 0.033 
0.026 0.028 0.039 0.038 1.000 0.030 0.030 
0.038 0.029 0.020 0.033 0.030 1.000 1.000 
0.038 0.029 0.020 0.033 0.030 1.000 1.000 


In [103]:
for i in range(num_documentos):
    similarity = cosine_similarity(vectores_tfidf_textos[i], vectores_tfidf_query[0])
    # print("{:.3f}".format(similarity), end=" ")  # Formatear y mostrar con tres decimales
    print(similarity)


0.9999999999999999
0.03274389468556651
0.01727734783562229
0.024843343082555657
0.025529804520065116
0.03839523054178518
0.03839523054178518
